In [20]:
## 복~습
## 단순선형회귀(simple linear regression)
## 데이터는 온도에 따른 오존량 
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats #science python : 과학연산을 위한 파이썬 모듈, 통계를 다루는 서브모듈 포함

# 1. data loading(raw data loading)
df = pd.read_csv("./data/ozone.csv")
#display(df)

# 2. Data Pre processing(필요데이터 선별, 결측치, 이상치제거)
# 2-1. 필요한 col부터 추출
df = df[["Ozone","Temp"]] #(153,2)

# 2-2. 결측치 처리 
df = df.dropna(how="any", inplace=False) #(116,2) #결측 보이면 다 삭제, 원본 훼손은 하지 말고  

# 2-3. 이상치 처리(IRQ기반 : Tukey Fences방식)
#이상치를 제대로 처리하지 않으면 가중치 때문에 학습에 크게 영향을 미칠 수 있다. 
#이상치가 일단 존재하는 지 확인 (boxplot사용)
'''
plt.boxplot(df["Temp"]) #이상치 미식별 (주석처리)
plt.boxplot(df["Ozone"]) #이상치 2건 식별(상위이상치) (주석처리)
'''
#IQR(= 3사분위값 - 1사분위값 )값을 구해서 상위, 하위 이상치의 경계를 계산 
q1,q3 = np.percentile(df["Ozone"],[25,75]) #tuple로 값을 리턴
iqr = q3-q1 #45.25
upper = q3 + iqr * 1.5 #131.125, 상위이상치를 처리하기 위한 경계값 반환 
mask = df["Ozone"] > upper #boolean mask 생성, broadcasting을 이용한 필터링 
df = df.loc[~mask] #걸러지지 않은것만 반환
display(df) # (114,2)

#2-4. 데이터가 선형적 관계에 있는지 산점도 도시 
#plt.scatter(df["Temp"],df["Ozone"]) #우상향 확인(주석처리)

#2-5. 정형화, 혹은 표준화를 이용해서 데이터의 scale변환  
# 정규화(Normalization) = (요소값-최소값) / (최대값 - 최대값)
# => 0~1 사이의 값으로 scale
# 표준화(Standardization) = (요소값-평균) / 표준편차 
# 정규화방식을 이용해서 데이터를 변환 
# 일반적으로는 모듈을 이용(다음시간에..), 우리는 프로그램으로 처리할 것 
df["Temp_Norm"] = (df["Temp"]-df["Temp"].min()) / (df["Temp"].max()-df["Temp"].min())
df["Ozone_Norm"] = (df["Ozone"]-df["Ozone"].min()) / (df["Ozone"].max()-df["Ozone"].min())
display(df) #(114,4)

##학습에 적합한 데이터의 완성 

,Ozone,Temp
0,41.0,67
1,36.0,72
2,12.0,74
3,18.0,62
5,28.0,66
...,...,...
147,14.0,63
148,30.0,70
150,14.0,75
151,18.0,76


,Ozone,Temp,Temp_Norm,Ozone_Norm
0,41.0,67,0.250,0.330579
1,36.0,72,0.375,0.289256
2,12.0,74,0.425,0.090909
3,18.0,62,0.125,0.140496
5,28.0,66,0.225,0.223140
...,...,...,...,...
147,14.0,63,0.150,0.107438
148,30.0,70,0.325,0.239669
150,14.0,75,0.450,0.107438
151,18.0,76,0.475,0.140496


In [34]:
#########################################    
## 우리가 텐서플로우로 얻은 값과 scipy로 얻은 값을(모듈을 사용해서 얻은 값을) 비교해보자 
## machine learning(Tensorflow)
# linear regression 시작
# 1. training data set 준비 
x_data = df["Temp_Norm"]
y_data = df["Ozone_Norm"]

# 2.필요노드들 생성하기 
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# 3. Weight, bias 
W = tf.Variable(tf.random_normal([1]), name ="weight")
b = tf.Variable(tf.random_normal([1]), name ="bias")

# 4. 가설(Hypothesis) 정의
H = W * x + b 

# 5. cost function 정의 
cost = tf.reduce_mean(tf.square(H-y))

# 6. train 
a = tf.Variable(0.0001)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# 7. session 초기화 작업 
init = tf.global_variables_initializer() #변수 초기화(초기변수 설정) 
sess = tf.Session()#세션 초기화 
sess.run(init)

# 8. 학습시작  
for step in range(30001):
    _,cost_val=sess.run([train,cost],  #cost값만 보기 
             feed_dict = {x : x_data,
                          y : y_data})
    if step % 300 ==0:
        print("cost값은 : {}".format(cost_val))
#print(sess.run(H, feed_dict = {x: [80]}))

# 이렇게 처리하면 문제가 있다. 
# 80이란 온도를 0~1사이의 값으로 scale(단위 맞춰주기)
tmp = (80 - df["Temp"].min()) / (df["Temp"].max() - df["Temp"].min())
result_scale = sess.run(H, feed_dict={x: tmp})
result = result_scale * (df["Ozone"].max()-df["Ozone"].min()) + df["Ozone"].min()
#result = result_scale * ( df["Ozone"].min()) / (df["Ozone"].max() - df["Ozone"])
print(result)




cost값은 : 7.738083839416504
cost값은 : 6.642667770385742
cost값은 : 5.703506946563721
cost값은 : 4.898297309875488
cost값은 : 4.20794677734375
cost값은 : 3.616058588027954
cost값은 : 3.1085894107818604
cost값은 : 2.673492908477783
cost값은 : 2.300447463989258
cost값은 : 1.9805984497070312
cost값은 : 1.7063558101654053
cost값은 : 1.471214771270752
cost값은 : 1.2695976495742798
cost값은 : 1.09671950340271
cost값은 : 0.948480486869812
cost값은 : 0.8213669657707214
cost값은 : 0.7123634815216064
cost값은 : 0.6188884973526001
cost값은 : 0.5387259125709534
cost값은 : 0.4699760675430298
cost값은 : 0.41101157665252686
cost값은 : 0.3604367971420288
cost값은 : 0.31705421209335327
cost값은 : 0.2798381745815277
cost값은 : 0.24790886044502258
cost값은 : 0.22051239013671875
cost값은 : 0.19700200855731964
cost값은 : 0.17682358622550964
cost값은 : 0.15950189530849457
cost값은 : 0.14462928473949432
cost값은 : 0.1318565458059311
cost값은 : 0.12088450789451599
cost값은 : 0.11145605146884918
cost값은 : 0.10335125029087067
cost값은 : 0.09638120234012604
cost값은 : 0.0903843045

In [31]:
# scipy의 stats module을 사용해서 편하게 처리 
x = df["Temp"]
y = df["Ozone"]

result = stats.linregress(x,y)
print(result)
print(80*result[0] + result[1])



LinregressResult(slope=2.344807849550286, intercept=-142.19084219133276, rvalue=0.7515377011359844, pvalue=5.802324426210489e-22, stderr=0.19448560867502498)
45.39378577269011


In [93]:
## multiple linear regression(다중 선형 회귀)
## 독립변수가 2개 이상인 경우 
## Hypothesis 가 변경 
## 단순 선형 회귀 => H = Wx + b 
## 다중 선형 회귀 => H = w1x1 + w2x2 + w3x3 + b (독립변수가 많아짐, weight도 따라 많아짐) 
## 행렬곱의 형태로서 값을 반환한다. (w와 x를 행렬로 표현) 
## 결국 matrix 형태로 W와 xrk 표현되요! 
## H = matmul(X,W) + b  #매트릭스의 곱 

## 퀴즈성적을 이용한 최종시험성적 예측 
## 사용하는 데이터 
## 퀴즈1 퀴즈2 퀴즈3 최종성적
##  73    80    75    152
##  93    88    93    185
##  89    91    90    180
##  96    98    100   196
##  73    66    70    142
##  80    90    50    ??

# 1. training data set 준비 
x_data = [[73,80,75],
          [93,88,93],
          [89,91,90],
          [96,98,100],
          [73,66,70]]    
y_data = [[152], [185], [180], [196] ,[142]] #다중회귀분석은 x와 y가 모두 2차원 matrix형태가 되어야 한다. 

# 2. placeholders 
#X = tf.placeholder(shape=[5,3], dtype=tf.float32) #shape지정 필요, 5행 3열 
X = tf.placeholder(shape=[None,3], dtype=tf.float32) #지정안하고 때에 다라 맞춰줄 수 있다. 
                                                     #학습할 때와 예측할 때와 다를 경우 
Y = tf.placeholder(shape=[None,1], dtype=tf.float32)         


# 3. Weight, bias 
W = tf.Variable(tf.random_normal([3,1]), name ="weight")
b = tf.Variable(tf.random_normal([1]), name ="bias")

# 4. 가설(Hypothesis) 정의
H = tf.matmul(X,W) + b

######### 이하동일

# 5. cost function 정의 
cost = tf.reduce_mean(tf.square(H-Y))

# 6. train 
a = tf.Variable(0.00001)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# 7. session 초기화 작업 
init = tf.global_variables_initializer() #변수 초기화(초기변수 설정) 
sess = tf.Session()#세션 초기화 
sess.run(init)

tmp = [80,90,50]

# 8. 학습시작  
for step in range(30001):
    _,cost_val=sess.run([train,cost],  #cost값만 보기 
             feed_dict = {X : x_data,
                          Y : y_data})
    if step % 300 ==0:
        print("cost값은 : {}".format(cost_val))
print(sess.run(H, feed_dict = {X:[tmp]}))




cost값은 : 168657.546875
cost값은 : 14.178178787231445
cost값은 : 12.105071067810059
cost값은 : 10.342670440673828
cost값은 : 8.844315528869629
cost값은 : 7.570440769195557
cost값은 : 6.487345218658447
cost값은 : 5.566341876983643
cost값은 : 4.783200263977051
cost값은 : 4.117182731628418
cost값은 : 3.5507140159606934
cost값은 : 3.0689029693603516
cost값은 : 2.659029483795166
cost값은 : 2.310288429260254
cost값은 : 2.0135326385498047
cost값은 : 1.7609403133392334
cost값은 : 1.54592764377594
cost값은 : 1.3628507852554321
cost값은 : 1.2068790197372437
cost값은 : 1.0740067958831787
cost값은 : 0.9607242345809937
cost값은 : 0.864152729511261
cost값은 : 0.7817274332046509
cost값은 : 0.7113808989524841
cost값은 : 0.6512568593025208
cost값은 : 0.599860668182373
cost값은 : 0.5558716654777527
cost값은 : 0.5181949734687805
cost값은 : 0.48588719964027405
cost값은 : 0.45812827348709106
cost값은 : 0.43426889181137085
cost값은 : 0.4136952757835388
cost값은 : 0.39594343304634094
cost값은 : 0.3805800676345825
cost값은 : 0.3672594130039215
cost값은 : 0.355666846036911
cost값은

In [99]:
#### 다중선형회귀 연습문제
### 관측된 Solar.R, Wind, Temp에 대한 Ozone량에 대한 데이터를 이용하여 machine learning을 진행하고
### Solar.R : 170, Wind : 7.4, Temp : 67.0에 대한 Ozone수치를 예측하세요.
### 27.9

import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats #science python : 과학연산을 위한 파이썬 모듈, 통계를 다루는 서브모듈 포함

# 1. data loading(raw data loading)
df = pd.read_csv("./data/ozone.csv")
display(df)

# 2. Data Pre processing(필요데이터 선별, 결측치, 이상치제거)
# 2-1. 필요한 col부터 추출
df = df[["Ozone","Solar.R","Wind","Temp"]] #(153,2)

# 2-2. 결측치 처리 
df = df.dropna(how="any", inplace=False) #(116,2) #결측 보이면 다 삭제, 원본 훼손은 하지 말고  

# 2-3. 이상치 처리(IRQ기반 : Tukey Fences방식)
#이상치를 제대로 처리하지 않으면 가중치 때문에 학습에 크게 영향을 미칠 수 있다. 
#이상치가 일단 존재하는 지 확인 (boxplot사용)

#plt.boxplot(df["Ozone"]) #이상치 2건 식별(상위이상치)
#plt.boxplot(df["Wind"]) # 이상치 3간식별(상위이상치)
#plt.boxplot(df["Solar.R"]) #이상치 미식별
#plt.boxplot(df["Temp"]) #이상치 미식별 


#IQR(= 3사분위값 - 1사분위값 )값을 구해서 상위, 하위 이상치의 경계를 계산 
q1,q3 = np.percentile(df["Ozone"],[25,75]) 
iqr = q3-q1 
upper = q3 + iqr * 1.5 
mask = df["Ozone"] > upper 
df = df.loc[~mask] 
#display(df) 

q1,q3 = np.percentile(df["Wind"],[25,75]) 
iqr = q3-q1 #45.25
upper = q3 + iqr * 1.5 
mask = df["Wind"] > upper 
df = df.loc[~mask] 
display(df) 

#2-4. 데이터가 선형적 관계에 있는지 산점도 도시 
'''
plt.scatter(df["Temp"],df["Ozone"]) #우상향 확인
plt.scatter(df["Wind"],df["Ozone"]) #우하향 확인
plt.scatter(df["Solar.R"],df["Ozone"])#우상향? 좀 에매함 
'''

#2-5. 정형화, 혹은 표준화를 이용해서 데이터의 scale변환  
# 정규화(Normalization) = (요소값-최소값) / (최대값 - 최대값)
# => 0~1 사이의 값으로 scale
# 표준화(Standardization) = (요소값-평균) / 표준편차 
# 정규화방식을 이용해서 데이터를 변환 
# 일반적으로는 모듈을 이용(다음시간에..), 우리는 프로그램으로 처리할 것 
df["Temp_Norm"] = (df["Temp"]-df["Temp"].min()) / (df["Temp"].max()-df["Temp"].min())
df["Solar.R_Norm"] = (df["Solar.R"]-df["Solar.R"].min()) / (df["Solar.R"].max()-df["Solar.R"].min())
df["Wind_Norm"] = (df["Wind"]-df["Wind"].min()) / (df["Wind"].max()-df["Wind"].min())
df["Ozone_Norm"] = (df["Ozone"]-df["Ozone"].min()) / (df["Ozone"].max()-df["Ozone"].min())
display(df) #(114,4)



##학습에 적합한 데이터의 완성 


# 1. training data set 준비 
x_data = df[["Solar.R_Norm","Wind_Norm","Temp_Norm"]] 
          
y_data = df[["Ozone_Norm"]] #다중회귀분석은 x와 y가 모두 2차원 matrix형태가 되어야 한다. 


# 2. placeholders 
#X = tf.placeholder(shape=[5,3], dtype=tf.float32) #shape지정 필요, 5행 3열 
X = tf.placeholder(shape=[None,3], dtype=tf.float32) #지정안하고 때에 다라 맞춰줄 수 있다. 
                                                     #학습할 때와 예측할 때와 다를 경우 
Y = tf.placeholder(shape=[None,1], dtype=tf.float32)         


# 3. Weight, bias 
W = tf.Variable(tf.random_normal([3,1]), name ="weight")
b = tf.Variable(tf.random_normal([1]), name ="bias")

# 4. 가설(Hypothesis) 정의
H = tf.matmul(X,W) + b

######### 이하동일

# 5. cost function 정의 
cost = tf.reduce_mean(tf.square(H-Y))

# 6. train 
a = tf.Variable(0.00001)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# 7. session 초기화 작업 
init = tf.global_variables_initializer() #변수 초기화(초기변수 설정) 
sess = tf.Session()#세션 초기화 
sess.run(init)

# 8. 학습시작  
#ex1 = [[170,7.4,67]]
tmp = [[(170 - df["Solar.R"].min()) / (df["Solar.R"].max() - df["Solar.R"].min()),
       (7.4 - df["Wind"].min()) / (df["Wind"].max() - df["Wind"].min()),
       (67 - df["Temp"].min()) / (df["Temp"].max() - df["Temp"].min())]]



for step in range(30001):
    _,cost_val=sess.run([train,cost],  #cost값만 보기 
             feed_dict = {X : x_data,
                          Y : y_data})
    if step % 300 ==0:
        print("cost값은 : {}".format(cost_val))

result_scale = sess.run(H, feed_dict={X: tmp})
result = result_scale * (df["Ozone"].max()-df["Ozone"].min()) + df["Ozone"].min() #역산, 정규화된거 원래수치로
print(result)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70,9,26
149,NaN,145.0,13.2,77,9,27
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


,Ozone,Solar.R,Wind,Temp
0,41.0,190.0,7.4,67
1,36.0,118.0,8.0,72
2,12.0,149.0,12.6,74
3,18.0,313.0,11.5,62
6,23.0,299.0,8.6,65
...,...,...,...,...
147,14.0,20.0,16.6,63
148,30.0,193.0,6.9,70
150,14.0,191.0,14.3,75
151,18.0,131.0,8.0,76


,Ozone,Solar.R,Wind,Temp,Temp_Norm,Solar.R_Norm,Wind_Norm,Ozone_Norm
0,41.0,190.0,7.4,67,0.230769,0.559633,0.356643,0.330579
1,36.0,118.0,8.0,72,0.358974,0.339450,0.398601,0.289256
2,12.0,149.0,12.6,74,0.410256,0.434251,0.720280,0.090909
3,18.0,313.0,11.5,62,0.102564,0.935780,0.643357,0.140496
6,23.0,299.0,8.6,65,0.179487,0.892966,0.440559,0.181818
...,...,...,...,...,...,...,...,...
147,14.0,20.0,16.6,63,0.128205,0.039755,1.000000,0.107438
148,30.0,193.0,6.9,70,0.307692,0.568807,0.321678,0.239669
150,14.0,191.0,14.3,75,0.435897,0.562691,0.839161,0.107438
151,18.0,131.0,8.0,76,0.461538,0.379205,0.398601,0.140496


cost값은 : 1.3388268947601318
cost값은 : 1.3138141632080078
cost값은 : 1.2893646955490112
cost값은 : 1.2654389142990112
cost값은 : 1.242056131362915
cost값은 : 1.2191712856292725
cost값은 : 1.1968066692352295
cost값은 : 1.174917221069336
cost값은 : 1.153522253036499
cost값은 : 1.1325856447219849
cost값은 : 1.1121219396591187
cost값은 : 1.0921037197113037
cost값은 : 1.0725198984146118
cost값은 : 1.0533747673034668
cost값은 : 1.0346384048461914
cost값은 : 1.0163359642028809
cost값은 : 0.9983981847763062
cost값은 : 0.9809019565582275
cost값은 : 0.96375572681427
cost값은 : 0.9470028877258301
cost값은 : 0.9306164979934692
cost값은 : 0.9145732522010803
cost값은 : 0.8989115357398987
cost값은 : 0.8835580945014954
cost값은 : 0.8685785531997681
cost값은 : 0.8539063930511475
cost값은 : 0.8395554423332214
cost값은 : 0.8255336880683899
cost값은 : 0.8117997050285339
cost값은 : 0.798383891582489
cost값은 : 0.7852623462677002
cost값은 : 0.7724055051803589
cost값은 : 0.7598649263381958
cost값은 : 0.7475793957710266
cost값은 : 0.7355571985244751
cost값은 : 0.723822474479675

In [94]:
### sklearn module(사이킷런, 텐서플로우 아님!)을 이용해서 학습시키고 예측까지 수행 
from sklearn import linear_model
import numpy as np 
import pandas as pd

df = pd.read_csv("./data/ozone.csv")
df = df[["Ozone","Solar.R","Wind","Temp"]]
df = df.dropna(how="any", inplace=False)

q1,q3 = np.percentile(df["Ozone"],[25,75]) 
iqr = q3-q1 
upper = q3 + iqr * 1.5 
mask = df["Ozone"] > upper 
df = df.loc[~mask] 
#display(df) 

q1,q3 = np.percentile(df["Wind"],[25,75]) 
iqr = q3-q1 #45.25
upper = q3 + iqr * 1.5 
mask = df["Wind"] > upper 
df = df.loc[~mask] 
display(df) 

#라이브러리가 알아서 처리해주기때문에 정규화 작업 생략 

###############################Machine learning
X = df[["Solar.R","Wind","Temp"]]
Y = df["Ozone"]

lm = linear_model.LinearRegression()
lm.fit(X,Y)

prediction = lm.predict([[170,7.4,67]]) #2차원 형태로 넣어줄것(X가 data frame)
print("예측값은 : {}" .format(prediction))

,Ozone,Solar.R,Wind,Temp
0,41.0,190.0,7.4,67
1,36.0,118.0,8.0,72
2,12.0,149.0,12.6,74
3,18.0,313.0,11.5,62
6,23.0,299.0,8.6,65
...,...,...,...,...
147,14.0,20.0,16.6,63
148,30.0,193.0,6.9,70
150,14.0,191.0,14.3,75
151,18.0,131.0,8.0,76


예측값은 : [27.97594664]


In [101]:
### 다중선형회귀 또다른 방법 -> 정규화 자동화  
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler #정규화를 시켜주는 서브모듈 포함

#data loading 
df = pd.read_csv("./data/ozone.csv")
df = df.dropna(how="any", inplace=False)

#전처리 
q1,q3 = np.percentile(df["Ozone"],[25,75]) 
iqr = q3-q1 
upper = q3 + iqr * 1.5 
mask = df["Ozone"] > upper 
df = df.loc[~mask] 
#display(df) 

q1,q3 = np.percentile(df["Wind"],[25,75]) 
iqr = q3-q1 #45.25
upper = q3 + iqr * 1.5 
mask = df["Wind"] > upper 
df = df.loc[~mask] 
display(df) 

# 정규화 
x_data = df[["Solar.R","Wind","Temp"]]
x_data = MinMaxScaler().fit_transform(x_data)
y_data = df["Ozone"].values.reshape(-1,1)
y_data = MinMaxScaler().fit_transform(y_data)

#######이하 생략 



,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
6,23.0,299.0,8.6,65,5,7
...,...,...,...,...,...,...
147,14.0,20.0,16.6,63,9,25
148,30.0,193.0,6.9,70,9,26
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


[[0.55963303 0.35664336 0.23076923]
 [0.33944954 0.3986014  0.35897436]
 [0.43425076 0.72027972 0.41025641]
 [0.93577982 0.64335664 0.1025641 ]
 [0.89296636 0.44055944 0.17948718]
 [0.28134557 0.8041958  0.02564103]
 [0.76146789 0.51748252 0.28205128]
 [0.86544343 0.48251748 0.20512821]
 [0.81651376 0.6013986  0.25641026]
 [0.17737003 0.76223776 0.        ]
 [1.         0.64335664 0.15384615]
 [0.91743119 0.67832168 0.20512821]
 [0.96330275 0.64335664 0.25641026]
 [0.11314985 0.51748252 0.1025641 ]
 [0.0030581  0.51748252 0.02564103]
 [0.95718654 1.         0.38461538]
 [0.05504587 0.51748252 0.07692308]
 [0.25993884 0.67832168 0.07692308]
 [0.01834862 0.67832168 0.23076923]
 [0.74923547 0.88111888 0.58974359]
 [0.66055046 0.23776224 0.53846154]
 [0.83180428 0.35664336 0.46153846]
 [0.36697248 0.51748252 0.61538462]
 [0.86850153 0.8041958  0.82051282]
 [0.96636086 0.64335664 0.74358974]
 [0.43119266 0.3986014  0.61538462]
 [0.56269113 0.88111888 0.48717949]
 [0.09174312 0.48251748 0.17

'\n<(다시) 지도학습> \n1. Linear regression(선형회귀)\n    - 단순선형회귀\n'